In [1]:
import os, sys
import numpy as np
from keras.models import Model
from keras.layers import Input, Dense, Flatten
from keras.layers import Conv1D, MaxPooling1D
from keras.layers import Embedding
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout, GlobalMaxPooling1D
print(" Python Library Imports Completed")

C:\Users\Prateek\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


 Python Library Imports Completed


In [2]:
import pandas as pd

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

In [3]:
# All Imports Complete, now importing the training data
df = pd.read_csv("train-1546603042473.csv", header=0)

In [4]:
df.head(5)

categories                                           converse
0  QUERIES FROM PHARMACY  please to verify instructions for drugname pat...
1        NEW APPOINTMENT  lmovm for patients mother to and schd rov trac...
2                 OTHERS  labtype and insurance approval other incoming ...
3                 OTHERS  clinical list changes medfusion secure electro...
4     MEDICATION RELATED  wants to wean off medication work phone name d...

In [ ]:
df.dtypes

In [ ]:
df['converse'].isnull().sum()

In [ ]:
df['categories'].isnull().sum()

In [ ]:
df.shape

In [6]:
# Will NOT remove records with null texts, but find their category, store it and replace Null with "EmptyConversation" keyword
#df.dropna(inplace=True)
df = df.replace(np.nan, 'emptyconversation', regex=True)
df.shape

(48699, 2)

In [7]:
df['converse'].isnull().sum()

0

In [8]:
# Find Categories for "emptyconversation"
pd.unique(df[(df.converse == 'emptyconversation')].categories)

array(['OTHERS', 'JUNK', 'CANCELLATION', 'RESCHEDULING'], dtype=object)

# EDA :)

In [ ]:
all_Categories = pd.unique(df.categories)
print(all_Categories, len(all_Categories))

In [ ]:

plt.figure(figsize=(20,8))
df.categories.value_counts().plot(kind='bar');

In [ ]:
# Counting words
word_Count = df['converse'].apply(lambda x: len(str(x).split(" ")))
word_Count.max()

### Padding size = 700, because testdata has max length of 652

In [9]:
x = df.iloc[:, 1].values
y = df.iloc[:, 0].values

In [10]:
x.shape

(48699,)

In [11]:
y.shape

(48699,)

In [ ]:
y

In [ ]:
# fetching unique classes:
# there are 21 unique classes for classification
print(np.unique(y, return_counts=True))

In [ ]:
unique_y=np.unique(y)
len(unique_y)

In [12]:
# Prepare tokenizer
tokenizer = Tokenizer(filters = '!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower=True)
# docs is list of strings that is passed to tokenizer.
tokenizer.fit_on_texts(x)

In [ ]:
# Getting all word list
#import nltk
#words = tokenizer.index_word.values()
#words

In [ ]:
#from nltk.stem import WordNetLemmatizer

#lemmatizer=WordNetLemmatizer()
#words = [lemmatizer.lemmatize(word) for word in words]


# This should be done at sentence level

### Not Attempting to remove stopwords - might loose keywords!

In [ ]:
#tokenizer.fit_on_texts(words)

In [ ]:
# integer encoding of words
#word_sequences = tokenizer.texts_to_sequences(words)
#print(words[1], word_sequences[1])

In [13]:
word_Index = tokenizer.word_index

vocab_Size = len(word_Index) + 1
print('Found %s unique tokens.' % vocab_Size)

Found 35749 unique tokens.


In [14]:
# integer encoding of words
sequences = tokenizer.texts_to_sequences(x)
print(x[1], sequences[1])

lmovm for patients mother to and schd rov tracy dominguez am kellee currie pm rhonda fanning [661, 5, 2, 518, 1, 3, 574, 106, 695, 815, 27, 380, 418, 25, 387, 2461]


In [ ]:
#len(word_sequences)
print(len(sequences))

In [ ]:
sequences

In [15]:
MAX_SEQUENCE_LENGTH = 700

padded_x = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

print('Shape of padded_sequence tensor:', padded_x.shape)

Shape of padded_sequence tensor: (48699, 700)


In [ ]:
print(padded_x)

In [16]:
 # Label Encoding categorical data for the classification category
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_Y = LabelEncoder()
y = labelencoder_Y.fit_transform(y)
print(y)
print(np.unique(y, return_counts=True))

[13  8  9 ... 15  8  7]
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20], dtype=int64), array([ 564,  127,   47,  811,  304,   18, 2253, 9010, 8907, 6253, 1043,
       1677,   91, 1464,  559, 8347, 1383,  590, 3018, 1212, 1021],
      dtype=int64))


In [17]:
# Perform one hot encoding 
y = to_categorical(y, num_classes= 21)
print(y)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [ ]:
#verifydf = pd.DataFrame(y)
#verifydf.to_csv("verify.csv")

In [18]:
print(padded_x.shape, y.shape)

(48699, 700) (48699, 21)


### Sampling through train_test_split

In [19]:
X_train, X_test, y_train, y_test = train_test_split(padded_x, y, test_size = 0.3, random_state = 21, stratify=y)

In [20]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(34089, 700)
(34089, 21)
(14610, 700)
(14610, 21)


In [19]:
y_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [21]:
# Preparing the Embedding layer
embeddings_index = {}
f = open(file="glove.6B.100d.txt", encoding="utf8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [22]:
embedding_Matrix = np.zeros((vocab_Size, 100))
for word, i in word_Index.items():
    embedding_Vector = embeddings_index.get(word)
    if embedding_Vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_Matrix[i] = embedding_Vector

print (embedding_Matrix.shape)

(35749, 100)


In [22]:
max_features = 1000
maxlen = 700
embedding_dims = 100
filters = 3
kernel_size = 5
hidden_dims = 256

# Model Building Starts

In [43]:
model2 = Sequential()
# Embedding layer maps our vocab indices into embedding_dims dimensions
model2.add(Embedding(vocab_Size, 
                    embedding_dims, 
                    weights=[embedding_Matrix], 
                    input_length=MAX_SEQUENCE_LENGTH, 
                    trainable=True))

In [44]:
from keras.optimizers import SGD
from keras.layers import core
from keras.optimizers import adadelta

In [45]:
#Convolution Layer
model2.add(Conv1D(filters, kernel_size, padding='valid', activation='relu', input_shape=(700, 100)))
model2.add(MaxPooling1D(pool_size=2, strides=None, padding='valid'))
model2.add(core.Dropout(0.2))
model2.add(core.Flatten())

# MLP Layer
model2.add(Dense(hidden_dims, activation='relu'))
model2.add(Dropout(0.2))
model2.add(Dense(21, activation='softmax'))

# Configuring model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
adaDelta = adadelta(lr=0.5, rho=0.95, epsilon=None, decay=0.0)
model2.compile(loss='categorical_crossentropy', optimizer='adaDelta', metrics=['accuracy'])

print(model2.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 700, 100)          3574900   
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 696, 3)            1503      
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 348, 3)            0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 348, 3)            0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 1044)              0         
_________________________________________________________________
dense_8 (Dense)              (None, 256)               267520    
_________________________________________________________________
dropout_8 (Dropout)          (None, 256)               0         
__________

In [47]:
# Training the model
#X_train, X_test, y_train, y_test 
model2.fit(X_train, y_train, batch_size=50,epochs=10,validation_data=(X_test, y_test))

Train on 34089 samples, validate on 14610 samples
Epoch 1/10
34089/34089 [==============================] - 653s 19ms/step - loss: 1.7520 - acc: 0.4770 - val_loss: 1.4660 - val_acc: 0.5553
Epoch 2/10
34089/34089 [==============================] - 641s 19ms/step - loss: 1.4477 - acc: 0.5516 - val_loss: 1.3303 - val_acc: 0.5836
Epoch 3/10
34089/34089 [==============================] - 642s 19ms/step - loss: 1.3314 - acc: 0.5839 - val_loss: 1.2609 - val_acc: 0.6077
Epoch 4/10
34089/34089 [==============================] - 649s 19ms/step - loss: 1.2681 - acc: 0.5998 - val_loss: 1.2646 - val_acc: 0.6144
Epoch 5/10
34089/34089 [==============================] - 242s 7ms/step - loss: 1.2133 - acc: 0.6183 - val_loss: 1.2024 - val_acc: 0.6230
Epoch 6/10
34089/34089 [==============================] - 209s 6ms/step - loss: 1.1826 - acc: 0.6283 - val_loss: 1.2026 - val_acc: 0.6304
Epoch 7/10
34089/34089 [==============================] - 208s 6ms/step - loss: 1.1504 - acc: 0.6375 - val_loss: 1.179

In [ ]:
from keras.optimizers import SGD
sgd = SGD(lr=0.01, decay=1e-6, momentum = 0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

In [ ]:
print(model.summary())

In [ ]:
# X_train, X_test, y_train, y_test
model.fit(X_train, y_train, batch_size=100, epochs=5, validation_data=(X_test, y_test))

# Working on Test Data

In [35]:
testData = pd.read_csv("test-1546603743049.csv", header=0)

In [36]:
testData.shape

(8581, 2)

In [37]:
testData = testData.replace(np.nan, 'emptyconversation', regex=True)

In [38]:
testData['converse'].isnull().sum()

0

In [ ]:
word_Count_test = testData['converse'].apply(lambda x: len(str(x).split(" ")))
word_Count_test.max()

In [39]:
testData = testData.iloc[:,1].values

In [40]:
from keras.preprocessing.text import Tokenizer
tokenizer2 = Tokenizer(filters = '!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower=True)
tokenizer2.fit_on_texts(testData)

In [41]:
word_Index1 = tokenizer2.word_index
vocab_Size1 = len(word_Index1) + 1

# integer encoding of words
sequences1 = tokenizer2.texts_to_sequences(testData)

MAX_SEQUENCE_LENGTH = 700
testData = pad_sequences(sequences1, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of padded_sequence tensor:', testData.shape)

Shape of padded_sequence tensor: (8581, 700)


In [69]:
predictions=model2.predict_classes(testData)

In [70]:
predictions

array([15,  9,  7, ...,  8,  7,  8], dtype=int64)

In [71]:
out = labelencoder_Y.inverse_transform(predictions)

C:\Users\Prateek\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [72]:
out.shape

(8581,)

In [73]:
output = pd.DataFrame(out)
output.to_csv("oput.csv")

#### As Concolution above gives 38% on test data
## Model 2  - MLP with 2 hidden layers

In [45]:
from keras.optimizers import adam
from keras.layers import core
model4 = Sequential()
# Embedding layer maps our vocab indices into embedding_dims dimensions
model4.add(Embedding(vocab_Size, 
                    100, 
                    weights=[embedding_Matrix], 
                    input_length=700, 
                    trainable=True))
# MLP Layer
model4.add(Dense(512, activation='relu'))
#model4.add(Dropout(0.4))
model4.add(core.Flatten())
#model4.add(Dense(64, activation='tanh'))
#model4.add(Dropout(0.4))
model4.add(Dense(21, activation='softmax'))

# Configuring model
adam = adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model4.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

print(model4.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 700, 100)          3574900   
_________________________________________________________________
dense_11 (Dense)             (None, 700, 512)          51712     
_________________________________________________________________
flatten_3 (Flatten)          (None, 358400)            0         
_________________________________________________________________
dense_12 (Dense)             (None, 21)                7526421   
Total params: 11,153,033
Trainable params: 11,153,033
Non-trainable params: 0
_________________________________________________________________
None


In [47]:

# Training Model 3
model4.fit(X_train, y_train, batch_size=64,epochs=2,validation_data=(X_test, y_test), class_weight='balanced')

Train on 34089 samples, validate on 14610 samples
Epoch 1/2
34089/34089 [==============================] - 580s 17ms/step - loss: 1.3968 - acc: 0.5907 - val_loss: 1.1123 - val_acc: 0.6678
Epoch 2/2
34089/34089 [==============================] - 550s 16ms/step - loss: 0.5909 - acc: 0.8054 - val_loss: 1.2576 - val_acc: 0.6592


In [48]:
predictions=model4.predict_classes(testData)
out = labelencoder_Y.inverse_transform(predictions)
output = pd.DataFrame(out)
output.to_csv("oput5.csv")

C:\Users\Prateek\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [ ]:
# Testing Data

In [42]:
predictions=model4.predict_classes(testData)
out = labelencoder_Y.inverse_transform(predictions)
output = pd.DataFrame(out)
output.to_csv("oput.csv")

C:\Users\Prateek\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


### LSTM Model

In [4]:

model3.summary()

C:\Users\Prateek\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


NameError: name 'Sequential' is not defined